# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [3]:
# Importar a biblioteca pandas
import pandas as pd

# O nome do arquivo que você baixou pode ser diferente.
# Ajuste o nome do arquivo no código abaixo.
# Exemplo: 'Consumo_Mensal_de_Energia_Elétrica_por_Classe_2024.xlsx'
caminho_do_arquivo = 'consumo.xlsx'

# Carregar os dados. Arquivos do governo brasileiro às vezes usam codificação 'latin-1'.
# Se for um arquivo Excel, talvez precise especificar a aba (sheet_name).
# Tente primeiro com pd.read_excel(). Se der erro, vamos ajustar.
try:
    # Tenta carregar como Excel. Pode haver várias abas, então vamos verificar.
    # Frequentemente, a aba com os dados se chama 'Consumo' ou algo similar.
    # Vamos supor que os dados estão na primeira aba (sheet_name=0) e o cabeçalho na primeira linha (header=0).
    df = pd.read_excel(caminho_do_arquivo, sheet_name=0, header=0)
except Exception as e:
    print(f"Erro ao ler o arquivo Excel: {e}")
    # Se falhar, podemos tentar ler como CSV
    # df = pd.read_csv(caminho_do_arquivo, encoding='latin-1', sep=';')


# Exibir as 5 primeiras linhas do nosso DataFrame para ver se carregou corretamente
print(df.head())

# Exibir informações gerais sobre o DataFrame (tipos de colunas, valores nulos)
print(df.info())

        Empresa de Pesquisa Energética - EPE Unnamed: 1 Unnamed: 2 Unnamed: 3  \
0  Consumo de energia elétrica na rede (MWh)        NaN        NaN        NaN   
1                            Sistema SIMPLES        NaN        NaN        NaN   
2                                        NaN        NaN        NaN        NaN   
3                                        NaN      2025*      2025*      2025*   
4                                        NaN        JAN        FEV        MAR   

  Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9  \
0        NaN        NaN        NaN        NaN        NaN        NaN   
1        NaN        NaN        NaN        NaN        NaN        NaN   
2        NaN        NaN        NaN        NaN        NaN        NaN   
3      2025*      2025*      2025*      2025*        NaN        NaN   
4        ABR        MAI        JUN        JUL        AGO        SET   

  Unnamed: 10 Unnamed: 11 Unnamed: 12 Unnamed: 13  
0         NaN         NaN         

In [4]:
import pandas as pd

# --- CARREGAR OS DADOS (como no passo anterior) ---
nome_do_arquivo = 'CONSUMO MENSAL DE ENERGIA ELÉTRICA POR CLASSE.xlsx - TOTAL.csv'
# Pulamos as linhas de cabeçalho do arquivo da EPE
df = pd.read_csv(nome_do_arquivo, skiprows=5)


# --- LIMPEZA E TRANSFORMAÇÃO ---

# 1. Renomear a primeira coluna para algo mais claro
#    O nome original pode ser 'Unnamed: 0' ou algo parecido.
df = df.rename(columns={df.columns[0]: 'localidade'})

# 2. Remover colunas vazias que o pandas pode ter criado no final
df = df.dropna(axis=1, how='all')

# 3. Transformar a tabela de 'wide' para 'long'
#    Isso vai transformar todas as colunas de data em linhas,
#    criando uma estrutura ideal para análise.
df_longo = pd.melt(df, id_vars=['localidade'], var_name='mes_ano', value_name='consumo')

# 4. Limpar a coluna 'mes_ano' e criar colunas separadas para 'ano' e 'mes'
#    A coluna 'mes_ano' vem com o formato 'JAN.2004', 'FEV.2004', etc.
df_longo[['mes', 'ano']] = df_longo['mes_ano'].str.split('.', expand=True)

# 5. Converter a coluna 'consumo' para um formato numérico, tratando erros.
#    'coerce' transforma qualquer valor que não seja número em 'NaN' (Not a Number)
df_longo['consumo'] = pd.to_numeric(df_longo['consumo'], errors='coerce')

# 6. Remover linhas onde o consumo seja nulo (geralmente linhas de totais ou rodapés)
df_longo = df_longo.dropna(subset=['consumo'])

# 7. Criar um mapa para converter os meses (texto) para número
mapa_meses = {
    'JAN': 1, 'FEV': 2, 'MAR': 3, 'ABR': 4, 'MAI': 5, 'JUN': 6,
    'JUL': 7, 'AGO': 8, 'SET': 9, 'OUT': 10, 'NOV': 11, 'DEZ': 12
}
df_longo['mes_num'] = df_longo['mes'].map(mapa_meses)

# 8. Criar uma coluna de data real (datetime)
#    Isso nos permite trabalhar com as datas de forma muito mais poderosa.
#    O ano e o mes_num precisam ser strings para serem concatenados.
df_longo['data'] = pd.to_datetime(df_longo['ano'].astype(str) + '-' + df_longo['mes_num'].astype(str) + '-01')

# 9. Limpeza final: selecionar e reordenar as colunas que vamos usar
df_final = df_longo[['data', 'localidade', 'consumo']].sort_values(by='data')


# --- VERIFICAR O RESULTADO ---

print("Transformação concluída! Veja como ficou a nova estrutura dos dados:")
print(df_final.head()) # Mostra as primeiras 5 linhas

print("\nE as informações do novo DataFrame:")
print(df_final.info()) # Mostra os tipos de dados e se há valores nulos

print("\nAlguns exemplos de localidades presentes na análise:")
print(df_final['localidade'].unique())

FileNotFoundError: [Errno 2] No such file or directory: 'CONSUMO MENSAL DE ENERGIA ELÉTRICA POR CLASSE.xlsx - TOTAL.csv'